In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from statistics import mean
%matplotlib inline

In [2]:
dataset = pd.read_csv('./Diabetes_Data_1999_2008.csv')
admission_src = pd.read_csv('./Admission_Source_Mapping.csv')
admission_type = pd.read_csv('./Admission_Type_Mapping.csv')
discharge_disposition = pd.read_csv('./Discharge_Disposition_Mapping.csv')

In [14]:
cleaned_df = dataset[dataset.readmitted == "NO"] #only looking at first encounters
cleaned_df = cleaned_df[cleaned_df.discharge_disposition_id != {18, 19, 20}]

admission_source = cleaned_df.iloc[:, 8:9]
timeHosp = cleaned_df.iloc[:, 9:10]
cleaned_df = cleaned_df.iloc[:, :19]
numMed = cleaned_df.iloc[:, 14:15]
demographics_df = cleaned_df.iloc[:, 2:5]
diag1_df = cleaned_df.iloc[:, 18:19]
cleaned_df = pd.concat([numMed, diag1_df, demographics_df, admission_source, timeHosp], axis = 1, join = 'inner')
cleaned_df = cleaned_df[cleaned_df.race != "?"]
cleaned_df = cleaned_df[cleaned_df.gender != "Unknown/Invalid"]

admin_srcMap = {}
admin_List = cleaned_df["admission_source_id"].value_counts().index.tolist()[:5]

for i, v in enumerate(admin_List):
    admin_srcMap[v] = i
for i in cleaned_df.admission_source_id:
    if i not in admin_srcMap:
        admin_srcMap[i] = 5

cleaned_df["admission_source_id"] = cleaned_df["admission_source_id"].map(admin_srcMap)

diag_1Map = {}
diag_1List = cleaned_df["diag_1"].value_counts().index.tolist()[:50]
diag_1Count = cleaned_df["diag_1"].value_counts().values.tolist()[:50]

for i, v in enumerate(diag_1List):
    diag_1Map[v] = i
for i in cleaned_df.diag_1:
    if i not in diag_1Map:
        diag_1Map[i] = 50

cleaned_df["diag_1"] = cleaned_df["diag_1"].map(diag_1Map)

print(cleaned_df)

        num_medications  diag_1             race  gender      age  \
0                     1      50        Caucasian  Female   [0-10)   
2                    13      50  AfricanAmerican  Female  [20-30)   
3                    16      45        Caucasian    Male  [30-40)   
4                     8      50        Caucasian    Male  [40-50)   
6                    21       0        Caucasian    Male  [60-70)   
...                 ...     ...              ...     ...      ...   
101759               15      21        Caucasian    Male  [80-90)   
101762               18      26  AfricanAmerican  Female  [80-90)   
101763                9      10        Caucasian    Male  [70-80)   
101764               21      11        Caucasian  Female  [80-90)   
101765                3      37        Caucasian    Male  [70-80)   

        admission_source_id  time_in_hospital  
0                         1                 1  
2                         0                 2  
3                         0

In [15]:
cleaned_df.to_csv("cleaned_data.csv", index = False)

In [25]:
cumMed = {}
for index, row in cleaned_df.iterrows():
    if row["diag_1"] not in cumMed:
        cumMed[row["diag_1"]] = [row["num_medications"]]
    else:
        cumMed[row["diag_1"]].append(row["num_medications"])
avgMed = {}
for i in cumMed:
    avgMed[i] = round(mean(cumMed[i]), 3)

{50: 14.979, 45: 14.041, 0: 20.55, 7: 12.084, 1: 16.295, 9: 14.691, 3: 20.716, 2: 11.328, 5: 13.974, 14: 15.339, 31: 15.96, 23: 19.083, 41: 11.203, 47: 15.876, 4: 16.283, 36: 12.393, 29: 17.352, 20: 17.598, 34: 19.711, 26: 13.664, 22: 13.795, 48: 15.881, 12: 17.826, 30: 14.833, 17: 19.853, 33: 9.519, 18: 19.724, 25: 11.22, 10: 17.498, 19: 14.375, 24: 14.193, 8: 10.879, 11: 18.922, 15: 13.431, 28: 10.317, 40: 19.627, 44: 18.992, 37: 12.99, 6: 21.411, 27: 16.64, 16: 13.399, 13: 11.654, 35: 17.546, 21: 10.909, 46: 12.092, 49: 12.702, 32: 19.323, 42: 11.888, 38: 12.539, 43: 15.749, 39: 16.739}


In [34]:
diagName = pd.read_table('./V26 I-9 Diagnosis.txt', )

In [38]:
print(diagName)

                        ID  Diagnosis
0      0010  CHOLERA D/T VIB CHOLERAE
1        0011  CHOLERA D/T VIB EL TOR
2                   0019  CHOLERA NOS
3                 0020  TYPHOID FEVER
4           0021  PARATYPHOID FEVER A
...                               ...
14020  V8902 SUS PLACENTL PROB NT FND
14021  V8903 SUS FETAL ANOMALY NT FND
14022  V8904 SUS FETAL GROWTH NOT FND
14023  V8905 SUS CERVCL SHORTN NT FND
14024    V8909 OTH SUS MAT/FTL NT FND

[14025 rows x 1 columns]
